In [3]:
library(mice)
library(finalfit)
library(ggplot2)
library(dplyr)
library(stats)
library(miceadds)
library(missMethods)
library(smotefamily)
library(caret)
library(cowplot)
library(mlr)
library(fastDummies)
library(DMwR2)
library(mlmi)
library(ranger)
library(scales)
library(Hmisc)

Warning message:
"pakiet 'mice' został zbudowany w wersji R 4.3.2"



Dołączanie pakietu: 'mice'


Następujący obiekt został zakryty z 'package:stats':

    filter


Następujące obiekty zostały zakryte z 'package:base':

    cbind, rbind


Warning message:
"pakiet 'ggplot2' został zbudowany w wersji R 4.3.3"

Dołączanie pakietu: 'dplyr'


Następujące obiekty zostały zakryte z 'package:stats':

    filter, lag


Następujące obiekty zostały zakryte z 'package:base':

    intersect, setdiff, setequal, union


* miceadds 3.16-18 (2023-01-06 10:54:00)

Ładowanie wymaganego pakietu: lattice

Warning message:
"pakiet 'cowplot' został zbudowany w wersji R 4.3.2"
Warning message:
"pakiet 'mlr' został zbudowany w wersji R 4.3.2"
Ładowanie wymaganego pakietu: ParamHelpers

Warning message:
"pakiet 'ParamHelpers' został zbudowany w wersji R 4.3.2"
Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} 

In [4]:
create_zero_df <- function(num_rows, num_cols, colnames) {
  #' Create dataframe of zeros.
  
  df <- as.data.frame(matrix(0, nrow = num_rows, ncol = num_cols))
  names(df) <- colnames
  return(df)
}

In [5]:
test_impute_narfcs <- function(data, m, maxit, method, formula, family, intcols, blots) {
    #' Compute statistical estimates for data imputed with narfcs.

    if (m > 1) {
        imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE, blots=blots)
        imp[intcols] <- imp[intcols] %>% lapply(as.integer)
        fit <- with(imp, glm(formula=as.formula(formula), family=family))
        tab <- summary(pool(fit), "all", conf.int = TRUE)
        tab <- tab[, c("term", "estimate", "2.5 %", "97.5 %")]
        tab <- as.data.frame(tab)
    }

    else {
        imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE, blots=blots) %>% complete()
        imp[intcols] <- imp[intcols] %>% lapply(as.integer)
        fit <- glm(formula=formula, family=family, data=imp)

        coefs <- as.data.frame(fit$coefficients)
        colnames(coefs) <- c("estimate")
        coefs <- cbind(term=rownames(coefs), coefs)
        rownames(coefs) <- NULL

        conf <- confint(fit)
        rownames(conf) <- NULL
        
        tab <- cbind(coefs, conf)
    }
    
    return(tab)
}

In [6]:
simulate_narfcs <- function(nsim, data, m, maxit, method, formula, family, intcols, blots, nvars) {
    #' Simulate statistical estimates computation for data imputed with narfcs.
    
    nrows <- nvars + 1

    tst_imp <- create_zero_df(nrows*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        start_index <- (n - 1) * nrows + 1
        end_index <- n * nrows
        tst_imp_n <- test_impute_narfcs(data_n, m, maxit, method, formula, family, intcols, blots)
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(tst_imp)
}

In [25]:
test_impute_ri <- function(data, m, maxit, method, formula, family, intcols, predm) {
    #' Compute statistical estimates for data imputed with random indicator.

    data$diabetes <- as.numeric(as.character(data$diabetes))
    lambda <- 2
    data$diaBP <- (data$diaBP^lambda - 1)/lambda

    imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE, predictorMatrix=predm) %>% complete()
    imp$diabetes <- rescale(imp$diabetes)
    imp$diabetes <- ifelse(imp$diabetes > 0.5, 1, 0)
    imp$diabetes <- as.factor(imp$diabetes)
    imp[intcols] <- imp[intcols] %>% lapply(as.integer)

    fit <- glm(formula=formula, family=family, data=imp)

    coefs <- as.data.frame(fit$coefficients)
    colnames(coefs) <- c("estimate")
    coefs <- cbind(term=rownames(coefs), coefs)
    rownames(coefs) <- NULL

    conf <- confint(fit)
    rownames(conf) <- NULL
    
    tab <- cbind(coefs, conf)
    
    return(tab)
}

In [8]:
simulate_ri <- function(nsim, data, m, maxit, method, formula, family, intcols, predm, nvars) {
    #' Simulate statistical estimates computation for data imputed with random indicator.

    nrows <- nvars + 1
    
    tst_imp <- create_zero_df(nrows*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        start_index <- (n - 1) * nrows + 1
        end_index <- n * nrows
        tst_imp_n <- test_impute_ri(data_n, m, maxit, method, formula, family, intcols, predm)
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(tst_imp)
}

In [9]:
PB <- function(sim_imp, real_coefs) {
    #' Compute percent bias.

    exp_est <- aggregate(sim_imp$estimate, list(sim_imp$term), FUN=mean)
    rownames(exp_est) <- exp_est$Group.1
    exp_est <- subset(exp_est, select=-1)

    terms <- unique(sim_imp$term)
    pbs <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        pbs[t] <- 100*abs((exp_est[terms[t],] - as.numeric(real_coefs[terms[t]])) / as.numeric(real_coefs[terms[t]]))
    }
    
    return(pbs)
}

In [10]:
CR <- function(sim_imp, real_coefs) {
    #' Compute coverage rate.

    terms <- unique(sim_imp$term)
    crs <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        tsubs <- subset(sim_imp, term==terms[t])
        crs[t] <- mean(tsubs["2.5 %"] < as.numeric(real_coefs[terms[t]]) & as.numeric(real_coefs[terms[t]]) < tsubs["97.5 %"])
    }

    return(crs)
}

In [11]:
AW <- function(sim_imp, real_coefs) {
    #' Compute average width of confidence intervals.

    terms <- unique(sim_imp$term)
    aws <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        tsubs <- subset(sim_imp, term==terms[t])
        diff <- tsubs["97.5 %"] - tsubs["2.5 %"]
        aws[t] <- mean(diff[, 1])
    }

    return(aws)   
}

In [12]:
csp1_exp <- 1.468584
csp2_exp <- 11.16717
blots <- list(diabetes = list(ums = as.character(csp1_exp)), diaBP = list(ums = as.character(csp2_exp)))

In [13]:
fram <- read.csv("Framingham.csv")
fram_mnar <- read.csv("fram_mnar2.csv")

In [14]:
fcols <- c("male", "education", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "TenYearCHD")
bcols <- c("male", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "TenYearCHD")
intcols <- c("age", "cigsPerDay", "glucose", "totChol", "heartRate")
fram_mnar[fcols] <- fram_mnar[fcols] %>% lapply(factor)
fram[fcols] <- fram[fcols] %>% lapply(factor)

In [15]:
glm_fit <- glm(formula = TenYearCHD ~ diabetes + diaBP, family = "binomial", data = fram)

In [16]:
real_coefs <- glm_fit$coefficients
real_coefs

(Intercept)   diabetes1       diaBP 
 -4.4802073   1.3261336   0.0322185

##### NARFCS

In [17]:
method1 <- rep("", ncol(fram_mnar))
method1[9] <- "mnar.logreg"
method1[12] <- "mnar.norm"

In [33]:
sim_imp1 <- simulate_narfcs(1000, fram_mnar, 10, 25, method1, "TenYearCHD ~ diabetes + diaBP", "binomial", intcols, blots, 2)

Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged event

In [34]:
pb1 <- PB(sim_imp1, real_coefs)
pb1

[1]  5.986148 29.435013  9.931223

In [35]:
cr1 <- CR(sim_imp1, real_coefs)
cr1

[1] 0.880 0.747 0.876

In [36]:
aw1 <- AW(sim_imp1, real_coefs)
aw1

[1] 1.62763314 1.13818083 0.01891911

##### Random indicator

In [37]:
predm <- make.predictorMatrix(fram_mnar)
predm["diaBP", "diabetes"] <- 0
predm["diabetes", "diaBP"] <- 0

In [38]:
method2 <- rep("", ncol(fram_mnar))
method2[c(9, 12)] <- "ri"

In [39]:
sim_imp2 <- simulate_ri(1000, fram_mnar, 1, 1, method2, "TenYearCHD ~ diabetes + diaBP", "binomial", intcols, predm, 2)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Warning message:
"Number of logged events: 1"
Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Warning message:
"Number of logged events: 1"
Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie p

In [40]:
pb2 <- PB(sim_imp2, real_coefs)
pb2

[1] 44.24935 14.68313 99.27271

In [41]:
cr2 <- CR(sim_imp2, real_coefs)
cr2

[1] 0.003 0.797 0.000

In [42]:
aw2 <- AW(sim_imp2, real_coefs)
aw2

[1] 1.141134922 0.929882621 0.000360093